In [1]:
#Inisialisasi
import GateLibrary2 as gl
from sympy import Matrix
import sympy as sp
from sympy.physics.quantum import Dagger, TensorProduct
from sympy import simplify,N
import numpy as np
import matplotlib.pyplot as plt

Parameter yang digunakan

In [2]:
#Definisikan control qubit
gamma = sp.Symbol('gamma',real = True)
rho_c = Matrix([[gamma,sp.sqrt(gamma*(1-gamma))],[sp.sqrt(gamma*(1-gamma)),1-gamma]])

#Definisikan transmitted qubit
a = sp.Symbol('a',real = True)
b = sp.Symbol('b',real = True)
c = sp.Symbol('c',real = True)
pauli0 = sp.Matrix([[1,0],[0,1]])
pauli1 = sp.Matrix([[0,1],[1,0]])
pauli2 = sp.Matrix([[0,-sp.I],[sp.I,0]])
pauli3 = sp.Matrix([[1,0],[0,-1]])
rho_t = sp.Rational(1/2)*pauli0 + (a/2)*pauli1 + (b/2)*pauli2 + (c/2)*pauli3


In [3]:
rho_t

Matrix([
[  c/2 + 1/2, a/2 - I*b/2],
[a/2 + I*b/2,   1/2 - c/2]])

In [4]:
#Kekuatan noise
NoiseStrength = 0.0
#NoiseStrength = np.arange(0.0,1.01,0.01)
#NoiseStrength = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]

#Nilai gamma
GammaValue = 1/2
GammaValue = simplify(GammaValue)

#Nilai parameter [a,b,c]
qubit_param = [1,0,0]


In [5]:
gate_p = gl.choose_gate('p',1)
gate_q = gl.choose_gate('p',3)

In [6]:
#Bentuk \rho_c \otimes \rho_t
total_qubit = TensorProduct(rho_c,rho_t)

rho0 = Matrix([[1,0],[0,0]])
rho1 = Matrix([[0,0],[0,1]])
#Bentuk K_{ij}
K = [[0 for i in range(4)] for i in range(4)]
K_dag = [[0 for i in range(4)] for i in range(4)]
for i in range(4):
    for j in range(4):
        pauli_ij = gate_p['matrices'][i]*gate_q['matrices'][j]
        pauli_ji = gate_p['matrices'][j]*gate_q['matrices'][i]
        K[i][j] = (TensorProduct(rho0,pauli_ij) + TensorProduct(rho1,pauli_ji))
        K_dag[i][j] = Dagger(TensorProduct(rho0,pauli_ij) + TensorProduct(rho1,pauli_ji))

In [7]:
K

[[Matrix([
  [1 - p,     0,     0,     0],
  [    0, 1 - p,     0,     0],
  [    0,     0, 1 - p,     0],
  [    0,     0,     0, 1 - p]]),
  Matrix([
  [0, 0,                   0,                   0],
  [0, 0,                   0,                   0],
  [0, 0,                   0, sqrt(p)*sqrt(1 - p)],
  [0, 0, sqrt(p)*sqrt(1 - p),                   0]]),
  Matrix([
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]]),
  Matrix([
  [sqrt(p)*sqrt(1 - p),                    0, 0, 0],
  [                  0, -sqrt(p)*sqrt(1 - p), 0, 0],
  [                  0,                    0, 0, 0],
  [                  0,                    0, 0, 0]])],
 [Matrix([
  [                  0, sqrt(p)*sqrt(1 - p), 0, 0],
  [sqrt(p)*sqrt(1 - p),                   0, 0, 0],
  [                  0,                   0, 0, 0],
  [                  0,                   0, 0, 0]]),
  Matrix([
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]]),
  Matrix([
  [0, 0, 0, 0],
  [0, 0, 0

In [8]:
K_dag

[[Matrix([
  [1 - p,     0,     0,     0],
  [    0, 1 - p,     0,     0],
  [    0,     0, 1 - p,     0],
  [    0,     0,     0, 1 - p]]),
  Matrix([
  [0, 0,                                         0,                                         0],
  [0, 0,                                         0,                                         0],
  [0, 0,                                         0, conjugate(sqrt(p))*conjugate(sqrt(1 - p))],
  [0, 0, conjugate(sqrt(p))*conjugate(sqrt(1 - p)),                                         0]]),
  Matrix([
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]]),
  Matrix([
  [conjugate(sqrt(p))*conjugate(sqrt(1 - p)),                                          0, 0, 0],
  [                                        0, -conjugate(sqrt(p))*conjugate(sqrt(1 - p)), 0, 0],
  [                                        0,                                          0, 0, 0],
  [                                        0,                                      

In [9]:
#Inisialisasi matrix untuk output state
output_state = Matrix([[0 for i in range(4)] for j in range(4)])
#Jumlahkan K_{ij} \rho_c\otimes\rho_t \K_{ij}^\dagger
for i in range(4):
    for j in range(4):
        output_state += K[i][j]*total_qubit*K_dag[i][j]


In [10]:
output_state

Matrix([
[        gamma*sqrt(p)*(1/2 - c/2)*sqrt(1 - p)*conjugate(sqrt(p))*conjugate(sqrt(1 - p)) + gamma*sqrt(p)*sqrt(1 - p)*(c/2 + 1/2)*conjugate(sqrt(p))*conjugate(sqrt(1 - p)) + gamma*p**2*(1/2 - c/2) + gamma*(1 - p)**2*(c/2 + 1/2), -gamma*sqrt(p)*sqrt(1 - p)*(a/2 - I*b/2)*conjugate(sqrt(p))*conjugate(sqrt(1 - p)) + gamma*sqrt(p)*sqrt(1 - p)*(a/2 + I*b/2)*conjugate(sqrt(p))*conjugate(sqrt(1 - p)) - gamma*p**2*(a/2 + I*b/2) + gamma*(1 - p)**2*(a/2 - I*b/2),                                                                                                                                                                                                            sqrt(gamma*(1 - gamma))*(1 - p)**2*(c/2 + 1/2),                                                                                                                                                                                                           sqrt(gamma*(1 - gamma))*(1 - p)**2*(a/2 - I*b/2)],
[gamma*sqrt(p)*sqrt(1 - p)*(a/2

Substitusi nilai dari p dan q

In [11]:
#Bagian ini perlu direvisi, kalau kasus keduanya sama strengthnya atau beda seperti apa
p = sp.Symbol('p',real = True)
#q = sp.Symbol('q',real = True)
output_subs = output_state.subs(p,NoiseStrength)
output_subs2 = output_subs
#output_subs2 = output_subs.subs(q,1/2)
#output_state
output_subs2

Matrix([
[                    gamma*(c/2 + 1/2),                   gamma*(a/2 - I*b/2),   sqrt(gamma*(1 - gamma))*(c/2 + 1/2), sqrt(gamma*(1 - gamma))*(a/2 - I*b/2)],
[                  gamma*(a/2 + I*b/2),                     gamma*(1/2 - c/2), sqrt(gamma*(1 - gamma))*(a/2 + I*b/2),   sqrt(gamma*(1 - gamma))*(1/2 - c/2)],
[  sqrt(gamma*(1 - gamma))*(c/2 + 1/2), sqrt(gamma*(1 - gamma))*(a/2 - I*b/2),               (1 - gamma)*(c/2 + 1/2),             (1 - gamma)*(a/2 - I*b/2)],
[sqrt(gamma*(1 - gamma))*(a/2 + I*b/2),   sqrt(gamma*(1 - gamma))*(1/2 - c/2),             (1 - gamma)*(a/2 + I*b/2),               (1/2 - c/2)*(1 - gamma)]])

In [12]:
#Substitusi nilai parameter, nanti harus direvisi agar ditentukan di awal fungsi
for i in range(4):
    output_subs2 = output_subs2.subs(a,qubit_param[0])
    output_subs2 = output_subs2.subs(b,qubit_param[1])
    output_subs2 = output_subs2.subs(c,qubit_param[2])
    output_subs2 = output_subs2.subs(gamma,GammaValue)

In [13]:
output_subs2

Matrix([
[0.25, 0.25, 0.25, 0.25],
[0.25, 0.25, 0.25, 0.25],
[0.25, 0.25, 0.25, 0.25],
[0.25, 0.25, 0.25, 0.25]])

Di bagian ini kita ingin mencari nilai dari $\tilde{\rho_c}$

In [14]:
#Bagian ini juga perlu direvisi, lebih baik melakukan partial trace dengan iterasi
#Cari rho_c_tilde
rho_c_tilde = Matrix([[output_subs2[0,0]+output_subs2[1,1],output_subs2[0,2]+output_subs2[1,3]],[output_subs2[2,0]+output_subs2[3,1],output_subs2[2,2]+output_subs2[3,3]]])


In [15]:
rho_c_tilde

Matrix([
[0.5, 0.5],
[0.5, 0.5]])

In [16]:
#Cari eigen value dari output state dan \tilde{\rho}_c
Eigen_value = output_subs2.eigenvals()
EigValC = rho_c_tilde.eigenvals()

In [17]:
Eigen_value

{0: 2, 1.00000000000000: 1, -4.92445252617321e-33: 1}

In [18]:
EigValC

{0: 1, 1.00000000000000: 1}

In [19]:
#Buat eigen value ke dalam array
keys = gl.get_keys(Eigen_value)
keys_c = gl.get_keys(EigValC)

In [20]:
keys

[0, 0, 1.00000000000000, -4.92445252617321e-33]

In [21]:
keys_c

[0, 1.00000000000000]

Dari bagian ini, kita dapat menghitung von Neumann entropynya sebagai berikut

$H = -\sum_i\lambda_i \ln \lambda_i$

In [22]:
#Hitung von Neumann entropy menggunakan nilai eigen
H_output = gl.vonNeumann(keys)
H_c = gl.vonNeumann(keys_c)

In [23]:
H_output

-3.66335415513154e-31 + 4.92445252617321e-33*I*pi

In [24]:
H_c

0

Sekarang kita akan menghitung nilai dari Holevo Capacitynya melalui ekspresi berikut
$\chi(\mathfrak{s}[\mathcal{N}_p,\mathcal{N}_q]) = 1 + H(\tilde{\rho}_c) - H^{min}(\mathfrak{s}[\mathcal{N}_p,\mathcal{N}_q])$

In [25]:
#Hitung Holevo capacity
Holevo_cap = 1 + N(H_c) - N(H_output)
Holevo_cap

1.0 - 1.54706238791775e-32*I